## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import urllib
import sys

# Import API key
sys.path.insert(1, '../')
from config import google_api_key

# Configure gmaps API key
gmaps.configure(api_key=google_api_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
filepath = os.path.join("../", "Weather_Database", "WeatherPy_Database.csv")
city_data_df = pd.read_csv(filepath)
city_data_df.head()

,name,country,lat,lng,max_temp,percent_humidity,percent_cloudiness,wind_speed,current_description
0,ushuaia,ar,-54.8000,-68.3000,51.80,87,75,5.75,broken clouds
1,nouakchott,mr,18.0858,-15.9785,59.00,33,0,9.22,clear sky
2,marsala,it,37.7966,12.4352,59.00,82,40,14.97,scattered clouds
3,pangnirtung,ca,66.1451,-65.7125,21.15,77,0,7.09,clear sky
4,miles city,us,46.4083,-105.8406,-7.60,64,75,5.59,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['max_temp'] <= max_temp) & (city_data_df['max_temp'] >= min_temp)]
preferred_cities_df

,name,country,lat,lng,max_temp,percent_humidity,percent_cloudiness,wind_speed,current_description
5,puerto ayora,ec,-0.7393,-90.3518,77.00,95,100,5.59,overcast clouds
9,atuona,pf,-9.8000,-139.0333,79.52,77,14,19.33,few clouds
19,rikitea,pf,-23.1203,-134.9692,77.56,70,69,12.75,broken clouds
22,soure,br,-0.7167,-48.5233,78.42,87,58,9.91,broken clouds
23,atuona,pf,-9.8000,-139.0333,79.52,77,14,19.33,few clouds
...,...,...,...,...,...,...,...,...,...
1787,mapastepec,mx,15.4333,-92.9000,75.51,58,0,3.36,clear sky
1793,rikitea,pf,-23.1203,-134.9692,77.58,69,73,11.86,broken clouds
1802,caravelas,br,-17.7125,-39.2481,75.94,91,100,10.76,light rain
1819,puerto ayacucho,ve,5.6639,-67.6236,77.50,83,100,1.39,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
pd.isna(preferred_cities_df)

,name,country,lat,lng,max_temp,percent_humidity,percent_cloudiness,wind_speed,current_description
5,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False
19,False,False,False,False,False,False,False,False,False
22,False,False,False,False,False,False,False,False,False
23,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
1787,False,False,False,False,False,False,False,False,False
1793,False,False,False,False,False,False,False,False,False
1802,False,False,False,False,False,False,False,False,False
1819,False,False,False,False,False,False,False,False,False


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["name", "country", "max_temp", "current_description", "lat", "lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,name,country,max_temp,current_description,lat,lng,Hotel Name
5,puerto ayora,ec,77.00,overcast clouds,-0.7393,-90.3518,
9,atuona,pf,79.52,few clouds,-9.8000,-139.0333,
19,rikitea,pf,77.56,broken clouds,-23.1203,-134.9692,
22,soure,br,78.42,broken clouds,-0.7167,-48.5233,
23,atuona,pf,79.52,few clouds,-9.8000,-139.0333,
...,...,...,...,...,...,...,...
1787,mapastepec,mx,75.51,clear sky,15.4333,-92.9000,
1793,rikitea,pf,77.58,broken clouds,-23.1203,-134.9692,
1802,caravelas,br,75.94,light rain,-17.7125,-39.2481,
1819,puerto ayacucho,ve,77.50,overcast clouds,5.6639,-67.6236,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_api_key
}

# 6b. Iterate through the hotel DataFrame.
for ind in hotel_df.index:

    # 6c. Get latitude and longitude from DataFrame
    lat = hotel_df['lat'][ind]
    lng = hotel_df['lng'][ind]
    latlng = str(lat) + ", " + str(lng)
    params['location'] = latlng
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    target_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?" + urllib.parse.urlencode(params)

    # 6e. Make request and retrieve the JSON data from the search. 
    response = requests.get(target_url).json()
    response
   
    try:
        # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotel_df.at[ind, 'Hotel Name'] = response['results'][0]['name']
    except:
        print("No results")
hotel_df
    
        

No results
No results
No results


,name,country,max_temp,current_description,lat,lng,Hotel Name
5,puerto ayora,ec,77.00,overcast clouds,-0.7393,-90.3518,Castillo Galapagos
9,atuona,pf,79.52,few clouds,-9.8000,-139.0333,Villa Enata
19,rikitea,pf,77.56,broken clouds,-23.1203,-134.9692,Pension Maro'i
22,soure,br,78.42,broken clouds,-0.7167,-48.5233,Casarão
23,atuona,pf,79.52,few clouds,-9.8000,-139.0333,Villa Enata
...,...,...,...,...,...,...,...
1787,mapastepec,mx,75.51,clear sky,15.4333,-92.9000,Hotel San Carlos
1793,rikitea,pf,77.58,broken clouds,-23.1203,-134.9692,Pension Maro'i
1802,caravelas,br,75.94,light rain,-17.7125,-39.2481,Pousada dos Navegantes
1819,puerto ayacucho,ve,77.50,overcast clouds,5.6639,-67.6236,Posada Turistica Manapiare


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset=['Hotel Name'])
clean_hotel_df

,name,country,max_temp,current_description,lat,lng,Hotel Name
5,puerto ayora,ec,77.00,overcast clouds,-0.7393,-90.3518,Castillo Galapagos
9,atuona,pf,79.52,few clouds,-9.8000,-139.0333,Villa Enata
19,rikitea,pf,77.56,broken clouds,-23.1203,-134.9692,Pension Maro'i
22,soure,br,78.42,broken clouds,-0.7167,-48.5233,Casarão
23,atuona,pf,79.52,few clouds,-9.8000,-139.0333,Villa Enata
...,...,...,...,...,...,...,...
1787,mapastepec,mx,75.51,clear sky,15.4333,-92.9000,Hotel San Carlos
1793,rikitea,pf,77.58,broken clouds,-23.1203,-134.9692,Pension Maro'i
1802,caravelas,br,75.94,light rain,-17.7125,-39.2481,Pousada dos Navegantes
1819,puerto ayacucho,ve,77.50,overcast clouds,5.6639,-67.6236,Posada Turistica Manapiare


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{name}</dd>
<dt>Country Code</dt><dd>{country}</dd>
<dt>Weather Description</dt><dd>{current_description}</dd>
<dt>Maxium Temperature</dt><dd>{max_temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["lat", "lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))